In [19]:
import ifcopenshell
import numpy as np
import os
from ifcopenshell import geom
import open3d as o3d

settings = geom.settings()
settings.set(settings.USE_WORLD_COORDS, True) 

In [20]:
import sys
import os
repo_root = os.path.abspath(os.path.join(os.getcwd(), '.', '.'))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [21]:
# Define a fixed color map for 13 labels
color_map = np.array([
    [1.0, 0.0, 0.0],  # Label 0: Red,  'ceiling'
    [0.0, 1.0, 0.0],  # Label 1: Green, 'floor'
    [0.0, 0.0, 1.0],  # Label 2: Blue,  'wall'
    [1.0, 1.0, 0.0],  # Label 3: Yellow, 'beam'
    [1.0, 0.0, 1.0],  # Label 4: Magenta, 'column'
    [0.0, 1.0, 1.0],  # Label 5: Cyan, 'window'
    [0.5, 0.5, 0.5],  # Label 6: Gray, 'door'
    [1.0, 0.5, 0.0],  # Label 7: Orange, 'chair'
    [0.5, 0.0, 1.0],  # Label 8: Purple, 'table'
    [0.5, 1.0, 0.5],  # Label 9: Light Green, 'bookcase'
    [0.5, 0.5, 1.0],  # Label 10: Light Blue, 'sofa'
    [1.0, 0.5, 0.5],  # Label 11: Pink, 'board'
    [0.0, 0.0, 0.0]   # Label 12: Black, 'clutter'
    ])

In [27]:
ifc_label_map = {
  "IfcCovering": 0, # "ceiling"
  "IfcSlab": 1, # "floor"
  #"IfcStair": 1,
  "IfcWallStandardCase": 2, # "wall"
  "IfcBeam": 3, # "beam"
  "IfcColumn": 4, # "column"
  "IfcWindow": 5, # "window"
  "IfcDoor": 6, # "door"
  "IfcBuildingElementProxy": 12, # "clutter"
  "IfcDistributionElement": 12, #
  "IfcFurnishingElement": 12, 
}

In [28]:
# Load IFC file
script_dir = os.path.dirname(os.getcwd())
ifc_file_path = os.path.join(script_dir, '.', 'docs', 'smartLab.ifc')

ifc_file = ifcopenshell.open(ifc_file_path) 

# Get all building elements (e.g., walls, doors)
elements = []
for tp in ifc_label_map.keys():
    elements += ifc_file.by_type(tp)

In [31]:
stair = ifc_file.by_type("IfcStair")
print(stair)

[#480708=IfcStair('3orDiQyEPBwxO8bTTuym2D',#19,'Stairs:Stairs:1130885',$,'Stairs:Stairs',#480707,#480705,'1130885',.NOTDEFINED.), #480710=IfcStair('3orDiQyEPBwxO8bTLuym2D',#19,'Stairs:Stairs:1130885',$,'Stairs:Stairs',#480704,$,'1130885',.NOTDEFINED.)]


In [29]:
def sample_points_on_mesh(verts, faces, spacing_mm=10):
    """
    Sample points on a mesh with a target spacing (e.g., 10mm between points).
    - verts: Array of shape (N, 3) containing mesh vertices.
    - faces: Array of shape (M, 3) containing triangular face indices.
    - spacing_mm: Desired distance between points in millimeters.
    Returns: Sampled points (shape: [K, 3]).
    """
    # Convert faces to NumPy array if it's a tuple
    if isinstance(faces, tuple):
        faces = np.array(faces)
    
    # Reshape to 2D if needed
    if len(faces.shape) == 1:
        faces = faces.reshape(-1, 3)

    triangles = verts[faces]
    
    # Calculate triangle areas (in mm²)
    vec1 = triangles[:, 1] - triangles[:, 0]
    vec2 = triangles[:, 2] - triangles[:, 0]
    areas = 0.5 * np.linalg.norm(np.cross(vec1, vec2), axis=1)

    # Calculate total surface area and number of points needed
    total_area_mm2 = np.sum(areas)
    points_per_mm2 = 1 / (spacing_mm ** 2)
    num_points = int(total_area_mm2 * points_per_mm2)
    num_points = max(num_points, 1)  # Ensure ≥1 point

    # Sample triangles weighted by their area
    probs = areas / areas.sum()
    sampled_tri_indices = np.random.choice(len(faces), size=num_points, p=probs)
    sampled_tris = triangles[sampled_tri_indices]

    # Barycentric coordinate sampling
    u = np.random.rand(num_points, 1)
    v = np.random.rand(num_points, 1)
    mask = (u + v) > 1
    u[mask] = 1 - u[mask]
    v[mask] = 1 - v[mask]
    w = 1 - (u + v)

    # Compute final points
    sampled_points = (u * sampled_tris[:, 0]) + (v * sampled_tris[:, 1]) + (w * sampled_tris[:, 2])
    return sampled_points

In [30]:
points = []
labels = []

for element in elements:
    # Get geometry
    shape = geom.create_shape(settings, element)
    verts = shape.geometry.verts  # Vertex coordinates (flat list)
    faces = shape.geometry.faces  # Triangular faces (indices)
    
    # Reshape vertices into (N, 3) array
    verts = np.array(verts).reshape(-1, 3)
    
    # Generate points on the mesh surface (see Step 4)
    spacing_mm = 0.01  # Points every 10mm
    sampled_points = sample_points_on_mesh(verts, faces, spacing_mm=spacing_mm)
    points.extend(sampled_points)
    
    # Assign label
    label = ifc_label_map.get(element.is_a(), -1)  # Default label for unknown classes
    labels.extend([label] * len(sampled_points))

In [ ]:
def add_noise(points, noise_level=0.01):
    noise = np.random.randn(*points.shape) * noise_level
    return points + noise

noisy_points = add_noise(points)

In [13]:

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(color_map[labels])
o3d.visualization.draw_geometries([pcd])
#o3d.io.write_point_cloud("labeled_cloud.ply", pcd)

In [14]:
sim_pc_path = os.path.join(script_dir, '.', 'docs', 'smartLab_simulated.ply')
o3d.io.write_point_cloud(sim_pc_path, pcd)

True

In [32]:
from pprint import pprint
print(len(elements))
pprint(elements)




33
[#337=IfcWallStandardCase('1$r5rWYh98mwhKRHZG_Pyg',#19,'Basic Wall:Generic - 200mm:351395',$,'Basic Wall:Generic - 200mm',#321,#336,'351395'),
 #373=IfcWallStandardCase('1$r5rWYh98mwhKRHZG_Pzu',#19,'Basic Wall:Generic - 200mm:351473',$,'Basic Wall:Generic - 200mm',#361,#372,'351473'),
 #419=IfcWallStandardCase('1$r5rWYh98mwhKRHZG_Put',#19,'Basic Wall:Interior - 135mm Partition (2-hr):351678',$,'Basic Wall:Interior - 135mm Partition (2-hr)',#403,#418,'351678'),
 #467=IfcWallStandardCase('1$r5rWYh98mwhKRHZG_Psi',#19,'Basic Wall:Generic - 200mm:351781',$,'Basic Wall:Generic - 200mm',#455,#466,'351781'),
 #490=IfcWallStandardCase('1$r5rWYh98mwhKRHZG_PtW',#19,'Basic Wall:Generic - 200mm:351849',$,'Basic Wall:Generic - 200mm',#478,#489,'351849'),
 #513=IfcWallStandardCase('1$r5rWYh98mwhKRHZG_PqT',#19,'Basic Wall:Interior - 135mm Partition (2-hr):351892',$,'Basic Wall:Interior - 135mm Partition (2-hr)',#501,#512,'351892'),
 #572=IfcWallStandardCase('1$r5rWYh98mwhKRHZG_PoF',#19,'Basic Wall: